In [39]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request as req
import requests
import time
import re
from urllib.parse import urljoin
import os

In [40]:
each_url =('k1m', 'k2m', 'k3m', 'k1ma', 'kmb')
url = ("https://19ch.tv/km/km.html")

In [41]:
def Subject_Link_Get(self):

    url_x = (f"https://19ch.tv/km/{self}.html")

    response = req.urlopen(url_x)
    parse_html = BeautifulSoup(response, 'html.parser')
    a = parse_html.find_all('a')
    T = parse_html.find('h1').text
    
    return (a, T)

In [43]:
def Lesson_Link_Get(self):
    
    title_list = []
    pdf_list = []
    lesson_list = []

    for i in self:
        if 'href' in i.attrs:
            h = i.attrs['href']

            if re.match(x + '/.*pdf', h):
                temp_url_pdf = urljoin(url, h)
                if not temp_url_pdf in pdf_list:
                    pdf_list.append(temp_url_pdf)
                    
            if re.match(x + '/\w*.html', h):
                temp_url_lesson = urljoin(url, h)
                if not temp_url_lesson in lesson_list:
                        lesson_list.append(temp_url_lesson)
                if i.text != '':
                    title_list.append(i.text)
                    
    
    return (title_list, pdf_list, lesson_list)

In [44]:
def Download_PDF(title, pdf):
    
    for (t, p) in zip(title, pdf):
        req.urlretrieve(p, f'/Users/makidaisuke/Desktop/my_pands/数学/問題集/pdf_dir/{t}.pdf')

In [51]:
def Download_Answer(title, answer):
    
    answer_list = []
    
    for (t, a) in zip(title, answer):
        time.sleep(1)
        r = req.urlopen(a)
        ph = BeautifulSoup(r, 'html.parser')
        answer_link = ph.find_all('a', class_='pure-button pure-button-primary')
        for al in answer_link:
            if al.text == '葉一先生の解答':
                temp_url_answer = urljoin(a, al.attrs['href'])
                answer_list.append(temp_url_answer)
    return answer_list

In [ ]:
for x in each_url:
    a, T = Subject_Link_Get(x)
    title, pdf, lesson =  Lesson_Link_Get(a)
    answer = Download_Answer(title, lesson)
    Download_PDF(title, pdf)
    Download_Answer(title, lesson)

    df = pd.DataFrame({'Title': title,
                                       'Lesson': lesson,
                                       'Answer': answer,
                                       'Finish': False})
    df.to_csv(f'/Users/makidaisuke/Desktop/my_pands/数学/問題集/{T}.csv')
else:
    print('完了')